In [1]:
import graphlab
graphlab.canvas.set_target('ipynb')

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.4.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


### Load and explore data

In [5]:
song_data = graphlab.SFrame('song_data.gl/')

2016-03-30 10:04:06,407 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: C:\Users\claudio\AppData\Local\Temp\graphlab_server_1459328645.log.0


This non-commercial license of GraphLab Create is assigned to c.bogazzi@gmail.com and will expire on March 04, 2017. For commercial licensing options, visit https://dato.com/buy/.


In [6]:
song_data.head()

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


### Compute the number of unique users for each of these artists: 'Kanye West', 'Foo Fighters', 'Taylor Swift' and 'Lady GaGa'.

In [9]:
# number of unique users who listened to songs by 'Kanye West'
users_west = song_data[song_data['artist'] == 'Kanye West']['user_id'].unique()
users_gaga = song_data[song_data['artist'] == 'Lady GaGa']['user_id'].unique()
users_ff = song_data[song_data['artist'] == 'Foo Fighters']['user_id'].unique()
users_taylor = song_data[song_data['artist'] == 'Taylor Swift']['user_id'].unique()

In [11]:
print len(users_west)
print len(users_gaga)
print len(users_ff)
print len(users_taylor)

2522
2928
2055
3246


### Using groupby-aggregate to find the most popular and least popular artist

In [12]:
song_data.groupby(key_columns='artist', operations={'total_count': graphlab.aggregate.SUM('listen_count')}).sort('total_count', ascending=False)

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


###  Using groupby-aggregate to find the most recommended songs

In [13]:
train_data,test_data = song_data.random_split(.8,seed=0)

In [14]:
personalized_model = graphlab.item_similarity_recommender.create(train_data,
                                                                user_id='user_id',
                                                                item_id='song')

Recsys training: model = item_similarity

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 7000            | 2.16147         |

| 8000            | 2.22062         |

| 9000            | 2.3093          |

+-----------------+-----------------+

Finished training in 2.62977s

In [15]:
subset_test_users = test_data['user_id'].unique()[0:10000]

In [18]:
subset_test_users_reco = personalized_model.recommend(subset_test_users,k=1)

recommendations finished on 1000/10000 queries. users per second: 2704.51

recommendations finished on 2000/10000 queries. users per second: 2838.38

recommendations finished on 3000/10000 queries. users per second: 2838.17

recommendations finished on 4000/10000 queries. users per second: 2824.27

recommendations finished on 5000/10000 queries. users per second: 2842.28

recommendations finished on 6000/10000 queries. users per second: 2842.09

recommendations finished on 7000/10000 queries. users per second: 2742.95

recommendations finished on 8000/10000 queries. users per second: 2768.5

recommendations finished on 9000/10000 queries. users per second: 2794.93

recommendations finished on 10000/10000 queries. users per second: 2786.34

In [19]:
most_recommended_song_by_personization_model = subset_test_users_reco.groupby(key_columns='song', operations={'total_count': graphlab.aggregate.COUNT()})

In [20]:
most_recommended_song_by_personization_model.sort('total_count', ascending=False)

song,total_count
Undo - Björk,433
Secrets - OneRepublic,390
Revelry - Kings Of Leon,235
You're The One - DwightYoakam ...,167
Fireflies - CharttraxxKaraoke ...,125
Hey_ Soul Sister - Train,102
Horn Concerto No. 4 in Eflat K495: II. Romance ...,98
Sehr kosmisch - Harmonia,74
OMG - Usher featuringwill.i.am ...,58
Dog Days Are Over (RadioEdit) - Florence + The ...,54


In [21]:
most_recommended_song_by_personization_model.sort('total_count', ascending=True)

song,total_count
Lentil - Sia,1
Ex Lover - Friendly Fires,1
So Nice So Smart - KimyaDawson ...,1
Wuthering Heights - KateBush ...,1
Shadow Journal - MaxRichter - Tilda Swinton ...,1
The Way Things Go -Octopus Project ...,1
When You Come Back Down -Nickel Creek ...,1
One More Chance (AlbumVersion) - The Notorious ...,1
Blue Shoes - Katie Melua,1
So Payaso (Version 2004)- Extremoduro ...,1
